In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import prince
import pprint
from datetime import date
import calendar
import warnings
warnings.filterwarnings("ignore") #ignore warnings

In [27]:
train = pd.read_csv('C:/Users/Will Jiang/Desktop/Emory Desktop/ML/Final Project/Project Data/Train000.csv')
test = pd.read_csv('C:/Users/Will Jiang/Desktop/Emory Desktop/ML/Final Project/Project Data/Train001.csv')

In [28]:
train.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
test.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

In [32]:
df = pd.concat([train, test]).drop(['id'], axis=0)

In [33]:
# 建立一個將hour資料轉換為日期格式的function
def get_date(hour):
    y = '20'+str(hour)[:2]
    m = str(hour)[2:4]
    d = str(hour)[4:6]
    return y+'-'+m+'-'+d

# 建立weekday欄位，將hour轉換後填入
df['weekday'] = pd.to_datetime(df.hour.apply(get_date)).dt.dayofweek.astype(str)

# 建立一個將hour資料轉換為時段的function
def tran_hour(x):
    x = x % 100
    while x in [23,0]:
        return '23-01'
    while x in [1,2]:
        return '01-03'
    while x in [3,4]:
        return '03-05'
    while x in [5,6]:
        return '05-07'
    while x in [7,8]:
        return '07-09'
    while x in [9,10]:
        return '09-11'
    while x in [11,12]:
        return '11-13'
    while x in [13,14]:
        return '13-15'
    while x in [15,16]:
        return '15-17'
    while x in [17,18]:
        return '17-19'
    while x in [19,20]:
        return '19-21'
    while x in [21,22]:
        return '21-23'

# 將hour轉換為時段
df['hour'] = df.hour.apply(tran_hour)

In [34]:
# 確認資料型別
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999998 entries, 0 to 999998
Data columns (total 24 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   click             int64 
 1   hour              object
 2   C1                int64 
 3   banner_pos        int64 
 4   site_id           object
 5   site_domain       object
 6   site_category     object
 7   app_id            object
 8   app_domain        object
 9   app_category      object
 10  device_id         object
 11  device_ip         object
 12  device_model      object
 13  device_type       int64 
 14  device_conn_type  int64 
 15  C14               int64 
 16  C15               int64 
 17  C16               int64 
 18  C17               int64 
 19  C18               int64 
 20  C19               int64 
 21  C20               int64 
 22  C21               int64 
 23  weekday           object
dtypes: int64(13), object(11)
memory usage: 381.5+ MB


In [35]:
len_of_feature_count = []
for i in df.columns[2:23].tolist():
    print(i, ':', len(df[i].astype(str).value_counts()))
    len_of_feature_count.append(len(df[i].astype(str).value_counts()))

C1 : 7
banner_pos : 7
site_id : 3012
site_domain : 3463
site_category : 21
app_id : 3960
app_domain : 259
app_category : 29
device_id : 280711
device_ip : 944568
device_model : 5782
device_type : 5
device_conn_type : 4
C14 : 2253
C15 : 8
C16 : 9
C17 : 403
C18 : 4
C19 : 65
C20 : 166
C21 : 55


In [36]:
# 建立一個list，將需要轉換行別的特徵名稱存入該list
need_tran_feature = df.columns[2:4].tolist() + df.columns[13:23].tolist()

# 依序將變數轉換為object型別
for i in need_tran_feature:
    df[i] = df[i].astype(str)

In [37]:
obj_features = []

for i in range(len(len_of_feature_count)):
    if len_of_feature_count[i] > 10:
        obj_features.append(df.columns[2:23].tolist()[i])
obj_features

['site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'C14',
 'C17',
 'C19',
 'C20',
 'C21']

In [38]:
df_describe = df.describe()
df_describe

,click
count,1.999998e+06
mean,1.701407e-01
std,3.757564e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.000000e+00


In [40]:
def obj_clean(X):
    # 定義一個縮減資料值的function，每次處理一個特徵向量

    def get_click_rate(x):
        # 定義一個取得點擊率的function
        temp = train[train[X.columns[0]] == x]
        res = round((temp.click.sum() / temp.click.count()),3)
        return res

    def get_type(V, str):
        # 定義一個取得新資料值之級距判斷的function
        very_high = df_describe.loc['mean','click'] + 0.04
        higher = df_describe.loc['mean','click'] + 0.02
        lower = df_describe.loc['mean','click'] - 0.02
        very_low = df_describe.loc['mean','click'] - 0.04

        vh_type = V[V[str] > very_high].index.tolist()
        hr_type = V[(V[str] > higher) & (V[str] < very_high)].index.tolist()
        vl_type = V[V[str] < very_low].index.tolist()
        lr_type = V[(V[str] < lower) & (V[str] > very_low)].index.tolist()

        return vh_type, hr_type, vl_type, lr_type

    def clean_function(x):
        # 定義一個依據級距轉換資料值的function
        # 判斷之依據為：總平均點擊率的正負  4% 為very_high(low), 總平均點擊率的正負 2％為higher (lower)
        while x in type_[0]:
            return 'very_high'
        while x in type_[1]:
            return 'higher'
        while x in type_[2]:
            return 'very_low'
        while x in type_[3]:
            return 'lower'
        return 'mid'
        
    print('Run: ', X.columns[0])
    fq = X[X.columns[0]].value_counts()
    # 建立一個暫存的資料值頻率列表
    # 理論上，將全部的資料值都進行分類轉換，可得到最佳效果；實務上為了執行時間效能，將捨去頻率低於排名前1000 row以後的資料值。
    if len(fq) > 1000:
        fq = fq[:1000]

    # 將頻率列表轉換為dataframe，並將index填入一個新的欄位。
    fq = pd.DataFrame(fq)
    fq['new_column'] = fq.index    

    # 使用index叫用get_click_rate function，取得每個資料值的點擊率
    fq['click_rate'] = fq.new_column.apply(get_click_rate)

    # 叫用 get_type function取得分類級距，並儲存為一個list，以便提供給下一個clean_function使用
    type_ = get_type(fq, 'click_rate')

    # 叫用 clean_funtion funtion，回傳轉換後的特徵向量
    return X[X.columns[0]].apply(clean_function)

In [53]:
# 使用for 迴圈將需轉換的特徵輸入到 obj_clean function
for i in obj_features:    
    df[[i]] = pd.DataFrame(obj_clean(df[[i]]))

df

Run:  site_id
Run:  site_domain
Run:  site_category
Run:  app_id
Run:  app_domain
Run:  app_category
Run:  device_id
Run:  device_ip
Run:  device_model
Run:  C14
Run:  C17
Run:  C19
Run:  C20
Run:  C21


,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,weekday
0,0,19-21,1010,1,very_low,very_low,very_low,very_low,higher,very_low,...,3,mid,320,50,mid,0,mid,mid,mid,1
1,0,15-17,1005,0,higher,higher,higher,higher,higher,higher,...,0,mid,320,50,mid,0,mid,mid,mid,5
2,0,09-11,1005,1,very_low,very_low,very_low,higher,higher,higher,...,0,mid,320,50,mid,3,mid,mid,mid,4
3,0,15-17,1005,0,very_low,very_low,very_low,mid,mid,very_low,...,0,mid,320,50,mid,3,mid,mid,mid,3
4,0,05-07,1005,0,higher,higher,higher,higher,higher,higher,...,0,mid,320,50,mid,0,mid,mid,mid,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,0,15-17,1005,1,lower,lower,mid,higher,higher,higher,...,0,mid,320,50,mid,3,mid,mid,mid,5
999995,0,23-01,1005,0,higher,higher,mid,higher,higher,higher,...,0,mid,320,50,mid,3,mid,mid,mid,4
999996,0,21-23,1005,0,higher,higher,higher,higher,higher,higher,...,0,mid,320,50,mid,0,mid,mid,mid,1
999997,0,13-15,1005,0,higher,higher,higher,higher,higher,higher,...,0,mid,320,50,mid,0,mid,mid,mid,3


In [54]:
for i in df.columns:
    print(i,len(df[i].unique()))

click 2
hour 12
C1 7
banner_pos 7
site_id 5
site_domain 5
site_category 4
app_id 5
app_domain 5
app_category 5
device_id 5
device_ip 5
device_model 5
device_type 5
device_conn_type 4
C14 1
C15 8
C16 9
C17 1
C18 4
C19 1
C20 1
C21 1
weekday 7


In [55]:
df.drop(['C14', 'C17', 'C19', 'C20', 'C21'], axis=1, inplace=True)

In [56]:
df = pd.get_dummies(df,drop_first=True)

In [57]:
df.columns

Index(['click', 'hour_03-05', 'hour_05-07', 'hour_07-09', 'hour_09-11',
       'hour_11-13', 'hour_13-15', 'hour_15-17', 'hour_17-19', 'hour_19-21',
       'hour_21-23', 'hour_23-01', 'C1_1002', 'C1_1005', 'C1_1007', 'C1_1008',
       'C1_1010', 'C1_1012', 'banner_pos_1', 'banner_pos_2', 'banner_pos_3',
       'banner_pos_4', 'banner_pos_5', 'banner_pos_7', 'site_id_lower',
       'site_id_mid', 'site_id_very_high', 'site_id_very_low',
       'site_domain_lower', 'site_domain_mid', 'site_domain_very_high',
       'site_domain_very_low', 'site_category_mid', 'site_category_very_high',
       'site_category_very_low', 'app_id_lower', 'app_id_mid',
       'app_id_very_high', 'app_id_very_low', 'app_domain_lower',
       'app_domain_mid', 'app_domain_very_high', 'app_domain_very_low',
       'app_category_lower', 'app_category_mid', 'app_category_very_high',
       'app_category_very_low', 'device_id_lower', 'device_id_mid',
       'device_id_very_high', 'device_id_very_low', 'device_ip_lo

In [67]:
train_len = len(train)
train = df[:train_len]
test = df[train_len:]

In [68]:
pre_X = train[train['click'] == 0].sample(n=len(train[train['click'] == 1]), random_state=111)
pre_X = pd.concat([pre_X, train[train['click'] == 1]]).sample(frac=1)
pre_y = pre_X[['click']]
pre_X.drop(['click'], axis=1, inplace=True)
test.drop(['click'], axis=1, inplace=True)

In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# 將新的資料集分割為訓練集與驗證集
pre_X_train, pre_X_test, pre_y_train, pre_y_test = train_test_split(pre_X, pre_y, test_size=0.20, stratify=pre_y, random_state=1)

In [ ]:
params = {"criterion":["gini", "entropy"], "max_depth":range(1,20)}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=params, scoring='neg_log_loss', cv=100, verbose=1, n_jobs=-1)
grid_search.fit(pre_X_train, pre_y_train)
grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_

In [ ]:
# 依據Grid Search的結果建立一個決策樹模型，並fit完整資料 (前置資料)
tree = grid_search.best_estimator_
tree.fit(pre_X,pre_y)

In [77]:
tree

DecisionTreeClassifier(max_depth=6)

In [78]:
# 輸出重要特徵，並依特徵之重要性排序
feature_importances = pd.DataFrame(tree.feature_importances_)
feature_importances.index = pre_X_train.columns
feature_importances = feature_importances.sort_values(0,ascending=False)
feature_importances

,0
site_id_very_low,0.466345
app_id_very_high,0.186034
site_id_very_high,0.095518
app_category_very_low,0.069853
app_id_very_low,0.058800
...,...
device_id_lower,0.000000
device_ip_lower,0.000000
device_ip_mid,0.000000
device_model_mid,0.000000


In [79]:
# 調整前置作業訓練集與驗證集，將特徵依特徵重要性縮減為重要性排名之1/3
pre_X_train = pre_X_train[feature_importances.index[:int(len(feature_importances)/3)]]
pre_X_test = pre_X_test[feature_importances.index[:int(len(feature_importances)/3)]]

In [80]:
pre_X_train

,site_id_very_low,app_id_very_high,site_id_very_high,app_category_very_low,app_id_very_low,C16_250,device_model_very_high,device_model_lower,C18_1,device_id_very_low,...,site_domain_very_high,banner_pos_7,site_category_very_high,device_ip_very_low,device_ip_very_high,device_conn_type_3,device_id_mid,app_domain_lower,device_conn_type_2,C16_50
543791,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
614847,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
103928,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
73841,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
40555,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942822,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1
857334,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
248910,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
513809,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [81]:
# 使用33％的重要特徵重新進行Grid Search調參
params = {"criterion":["gini", "entropy"], "max_depth":range(1,12)}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=params, scoring='neg_log_loss', cv=100, verbose=1, n_jobs=-1)
grid_search.fit(pre_X_train, pre_y_train)
grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_

Fitting 100 folds for each of 22 candidates, totalling 2200 fits


(-0.6067760984428198,
 DecisionTreeClassifier(max_depth=7),
 {'criterion': 'gini', 'max_depth': 7})

In [ ]:
# 調整前置作業完整資料集，將特徵依特徵重要性縮減為重要性排名之1/3
pre_X = pre_X[feature_importances.index[:int(len(feature_importances)/3)]]

# 依據Grid Search的結果建立一個決策樹模型，並fit完整資料 (前置資料)
tree = grid_search.best_estimator_
tree.fit(pre_X,pre_y)

In [88]:
# 輸出重要特徵，並依特徵之重要性排序
feature_importances = pd.DataFrame(tree.feature_importances_)
feature_importances.index = pre_X.columns
feature_importances = feature_importances.sort_values(0,ascending=False)
feature_importances

,0
site_id_very_low,0.484713
app_id_very_high,0.193362
site_id_very_high,0.099280
app_category_very_low,0.073712
app_id_very_low,0.061116
C16_250,0.025617
device_model_very_high,0.024923
C18_1,0.019846
device_model_lower,0.017432


In [89]:
# 最終預測模型之特徵，將採用特徵值 .005以上的變數
feature_len = len(feature_importances[feature_importances[feature_importances.columns[0]] > 0.005])

# 調整最終完整Train Set 與 Test set之特徵
y = train[['click']]
X = train[feature_importances[:feature_len].index]
test = test[feature_importances[:feature_len].index]

In [96]:
from xgboost import XGBClassifier

# 使用xgboost 建模，並指定先前調參得到的節點深度限制使用xgboost 建模，並指定先前調參得到的節點深度限制
model = XGBClassifier(tree_method = 'gpu_hist', n_jobs=-1, n_estimators=500, max_depth=11)
model.fit(X,y.values.ravel())
y_pred = model.predict_proba(X)

[11:26:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [97]:
print("log_loss: ",log_loss(y,y_pred))

log_loss:  0.4096210889192551


In [98]:
y_pred

array([[0.9469039 , 0.0530961 ],
       [0.76686215, 0.23313786],
       [0.8951936 , 0.10480645],
       ...,
       [0.8162542 , 0.18374579],
       [0.9469039 , 0.0530961 ],
       [0.9336424 , 0.06635762]], dtype=float32)